<a href="https://colab.research.google.com/github/Tsung-Hung/dummy-git/blob/master/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [219]:
import torch 
import torch.nn as nn
import torchvision 
import torch.nn.functional as F

# LayerNorm replace by BatchNorm

In [ ]:
#https://discuss.pytorch.org/t/f-batch-norm-returns-different-results-in-train-and-eval-mode-given-same-setup/140493
#https://stackoverflow.com/questions/68478856/pytorch-batchnorm2d-calculation
#https://www.bing.com/search?q=class+BatchNorm2d+source+code+&qs=n&form=QBRE&msbsrank=6_6__0&sp=-1&pq=class+batchnorm2d+source+&sc=6-25&sk=&cvid=08ABB45423784303AC7BE5FF2B9A5E71&ghsh=0&ghacc=0&ghpl=&ntref=1
#https://discuss.pytorch.org/t/how-to-use-scripting-with-custom-batchnorm/85375/5

In [80]:
batch, sentence_length, embedding_dim = 2, 3, 5
embedding = torch.randn(batch, sentence_length, embedding_dim)
print(embedding)
layer_norm = nn.LayerNorm(embedding_dim)
# Activate module
origin = layer_norm(embedding)
print(origin)

tensor([[[ 0.9641, -0.3264, -0.5704, -0.3200, -1.7319],
         [-1.0646, -0.6391, -0.4975, -1.2980, -0.5398],
         [-0.9724, -0.2570, -0.1610,  1.8681,  2.5809]],

        [[ 1.2818,  0.8586,  0.8603, -0.6301,  0.1373],
         [ 0.5544, -0.6314,  0.2906, -2.2502, -0.0423],
         [ 0.8799,  0.1243, -1.8196,  0.9262, -0.7537]]])
tensor([[[ 1.5874,  0.0822, -0.2023,  0.0897, -1.5570],
         [-0.8098,  0.5320,  0.9785, -1.5457,  0.8450],
         [-1.1604, -0.6364, -0.5661,  0.9204,  1.4425]],

        [[ 1.1558,  0.5289,  0.5314, -1.6765, -0.5397],
         [ 0.9710, -0.2158,  0.7070, -1.8360,  0.3738],
         [ 0.9665,  0.2424, -1.6206,  1.0108, -0.5991]]],
       grad_fn=<NativeLayerNormBackward0>)


In [59]:
def Layer_norm(x):
    '''
    BatchNorm2d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim, 1)
    layer_norm = nn.BatchNorm2d(x.shape[1], track_running_stats=True, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

def Layer_norm2(x):
    '''
    BatchNorm1d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim)
    layer_norm = nn.BatchNorm1d(x.shape[1], track_running_stats=False, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm(embedding)
test

tensor([[[ 0.3465, -1.1429, -1.0091,  1.5830,  0.2225],
         [-1.0241, -1.1517,  0.9735,  1.2919, -0.0896],
         [ 0.1474, -1.4635, -0.5604,  0.3211,  1.5554]],

        [[ 0.1451,  0.7724,  0.2184, -1.9261,  0.7903],
         [ 1.1391,  0.4523, -1.2866, -1.1000,  0.7952],
         [ 0.6111,  1.0796, -0.1115, -1.8388,  0.2597]]],
       grad_fn=<ReshapeAliasBackward0>)

In [77]:
batch, sentence_length, embedding_dim = embedding.shape
x = embedding.reshape(1, batch*sentence_length, embedding_dim, 1)
layer_norm = nn.BatchNorm2d(x.shape[1], track_running_stats=True, affine=True)
output = layer_norm(x)
output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
layer_norm.running_mean 

tensor([-0.0072, -0.0715, -0.0038, -0.0122, -0.0371, -0.0597])

In [52]:
def Layer_norm3(x):
    '''
    BatchNorm1d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim)
    layer_norm = nn.BatchNorm1d(x.shape[1], track_running_stats=True, affine=True)
    #print(layer_norm.weight)
    #print(layer_norm.bias)
    #print(layer_norm.running_mean)
    # Set the running statistics to constant
    layer_norm.running_mean.fill_(0)
    layer_norm.running_var.fill_(1) 

    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm3(embedding)
test

tensor([[[-0.3430, -1.5758,  0.3096,  1.5147,  0.0946],
         [-1.1580,  1.4034, -0.3166, -0.8549,  0.9261],
         [-1.3159, -1.0623,  1.1146,  0.4094,  0.8543]],

        [[ 0.6805, -0.5048,  0.2777,  1.2054, -1.6589],
         [-1.3882,  1.2785, -0.7990,  0.8942,  0.0144],
         [-0.0273, -0.7379, -1.3817,  1.4205,  0.7264]]],
       grad_fn=<ReshapeAliasBackward0>)

In [7]:
norm3 = nn.BatchNorm1d(3, track_running_stats=False, affine=True)
def loop_layernorm1(x):
    batch, sentence_length, embedding_dim = x.shape
    # layer_norm = nn.BatchNorm1d(sentence_length,momentum=1.0, affine=True, track_running_stats=False)
    
    split_x = torch.split(x,1,dim=0)
    concate_list = []
    for small_x in split_x:
        small_x = small_x.squeeze().permute(1,0)
        small_x = norm3(small_x).permute(1,0)
        concate_list.append(small_x)
        # print (small_x.shape)
    
    concate = torch.stack(concate_list)
    # print (concate.shape)
    return concate


test = loop_layernorm1(embedding)
test

tensor([[[ 0.6652, -0.7223,  1.6224, -0.4840, -1.0813],
         [ 0.2661,  1.4295,  0.0388, -1.6979, -0.0365],
         [-1.5133,  0.0102,  1.6431, -0.0839, -0.0561]],

        [[-1.6453,  1.1871,  0.2770,  0.7192, -0.5380],
         [-1.2470,  1.5584,  0.3359, -0.9127,  0.2655],
         [ 1.4374, -1.0207,  0.8625, -1.0459, -0.2332]]],
       grad_fn=<StackBackward0>)

In [ ]:
class LayerNorm2(nn.Module):
  def __init__(self, num_features, eps=1e-6):
    super().__init__()
    self.weight = nn.Parameter(torch.ones(num_features), requires_grad=False)
    self.bias = nn.Parameter(torch.zeros(num_features), requires_grad=False)
    self.eps = eps

  def forward(self, x):
    shape = x.shape
    mean = x.mean(-1)
    std = torch.sqrt(x.var(-1, unbiased=False))
    res = torch.stack([((x[i][j] - mean[i][j]).squeeze(0) / (std[i][j].squeeze(0) + self.eps)) for i in range(x.shape[0]) for j in range(x.shape[1])], dim=0)
    return res.reshape(shape)

layer_norm2 = LayerNorm2(embedding_dim)
# Activate module
test = layer_norm2(embedding)
test

In [247]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the SimpleNet model
class SimpleNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleNet, self).__init__()
        self.bn1 = nn.BatchNorm1d(10, track_running_stats=True, affine=True, momentum=1)
    
    def Layer_norm1(self, x):
        batch, sentence_length, embedding_dim = x.shape
        x = x.reshape(1, batch*sentence_length, embedding_dim)
        # layer_norm = nn.BatchNorm2d(x.shape[1], track_running_stats=False, affine=True)
        output = self.bn1(x)
        output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
        return output

    def forward(self, x):
        x = self.Layer_norm1(x) #input(1,10,64)
        x = x[0,0,0]
        return x

# Generate a simulated dataset
inputs = torch.rand(3, 10, 64)
labels = torch.randint(low=0, high=10, size=(3,))
inputs = torch.Tensor(inputs)
labels = torch.Tensor(labels).long()
train_dataset = torch.utils.data.TensorDataset(inputs, labels)

# Create a DataLoader to load and batch the data
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1)

# Define the loss function and the optimizer
net = SimpleNet()
criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(5):
    running_loss = 0.0
    print("1. Running mean:", net.bn1.running_mean)
    for i, data in enumerate(train_loader):
        inputs, labels = data
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print("2. Running mean:", net.bn1.running_mean)
    print('Epoch [{}/50], Loss: {:.4f}'.format(epoch+1, running_loss / len(train_loader)))
    print("3. Running mean:", net.bn1.running_mean)
    print("===========================")

1. Running mean: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
2. Running mean: tensor([0.4859, 0.5081, 0.5124, 0.4705, 0.4992, 0.5370, 0.4727, 0.5215, 0.4754,
        0.5214])
2. Running mean: tensor([0.5307, 0.4298, 0.4670, 0.4764, 0.4707, 0.5168, 0.5147, 0.5776, 0.5070,
        0.4452])
2. Running mean: tensor([0.4553, 0.5498, 0.5141, 0.5797, 0.4574, 0.4905, 0.4929, 0.5331, 0.4744,
        0.4797])
Epoch [1/50], Loss: 2.5307
3. Running mean: tensor([0.4553, 0.5498, 0.5141, 0.5797, 0.4574, 0.4905, 0.4929, 0.5331, 0.4744,
        0.4797])
1. Running mean: tensor([0.4553, 0.5498, 0.5141, 0.5797, 0.4574, 0.4905, 0.4929, 0.5331, 0.4744,
        0.4797])
2. Running mean: tensor([0.4859, 0.5081, 0.5124, 0.4705, 0.4992, 0.5370, 0.4727, 0.5215, 0.4754,
        0.5214])
2. Running mean: tensor([0.5307, 0.4298, 0.4670, 0.4764, 0.4707, 0.5168, 0.5147, 0.5776, 0.5070,
        0.4452])
2. Running mean: tensor([0.4553, 0.5498, 0.5141, 0.5797, 0.4574, 0.4905, 0.4929, 0.5331, 0.4744,
        0.47

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [248]:
print(net.bn1.weight)
print(net.bn1.bias)
print(net.bn1.running_mean)
print(net.bn1.running_var)

Parameter containing:
tensor([1.0887, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], requires_grad=True)
Parameter containing:
tensor([0.0785, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000], requires_grad=True)
tensor([0.4553, 0.5498, 0.5141, 0.5797, 0.4574, 0.4905, 0.4929, 0.5331, 0.4744,
        0.4797])
tensor([0.0992, 0.0833, 0.0768, 0.0835, 0.0836, 0.0877, 0.0653, 0.0674, 0.0726,
        0.0870])


In [249]:
def set_bn_training(m):
    if isinstance(m, nn.BatchNorm1d):
        m.train()

#test_sample = torch.randn(1, 10, 64)
#print(test_sample)
net.eval()
print("After Running mean:", net.bn1.running_mean)
print(net.bn1.training)

net.apply(set_bn_training)
output = net(test_sample)
print("Before Running mean:", net.bn1.running_mean)
print(net.bn1.training)
print(output)

After Running mean: tensor([0.4553, 0.5498, 0.5141, 0.5797, 0.4574, 0.4905, 0.4929, 0.5331, 0.4744,
        0.4797])
False
Before Running mean: tensor([-0.1496, -0.0035, -0.0163, -0.2303,  0.1592,  0.0254,  0.0513, -0.1963,
         0.0449,  0.0202])
True
tensor(1.6269, grad_fn=<SelectBackward0>)


In [250]:
torch.mean(test_sample, dim=2)

tensor([[-0.1496, -0.0035, -0.0163, -0.2303,  0.1592,  0.0254,  0.0513, -0.1963,
          0.0449,  0.0202]])

In [107]:
class SimpleNet_ln(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleNet_ln, self).__init__()
        self.ln1 = nn.LayerNorm(100)

    def forward(self, x):
        x = x.transpose(0,1)
        x = self.ln1(x)
        x = x.transpose(0,1)
        return x

# Define the loss function and the optimizer
net_ln = SimpleNet_ln()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        outputs = net_ln(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [{}/50], Loss: {:.4f}'.format(epoch+1, running_loss / len(train_loader)))

Epoch [1/50], Loss: 3.9652
Epoch [2/50], Loss: 3.9620
Epoch [3/50], Loss: 3.9658


# matmul 4D tensor

In [ ]:
a1 = torch.rand(2,2,4,8)
a2 = torch.rand(2,2,6,8)

result = torch.matmul(a1, a2.transpose(-2,-1))
print('Shape : ', result.shape)
result

Shape :  torch.Size([2, 2, 4, 6])


tensor([[[[2.7529, 1.8048, 3.4391, 3.1496, 2.6274, 3.4769],
          [2.4426, 2.0310, 2.9492, 2.6178, 2.0630, 3.0072],
          [2.5750, 1.7425, 2.7397, 2.7792, 2.1718, 3.1972],
          [1.6580, 1.1785, 2.3369, 1.8366, 1.7175, 2.1643]],

         [[1.8686, 2.8051, 1.5285, 2.3255, 2.7303, 1.7538],
          [1.8524, 3.0672, 2.7623, 2.5586, 2.9062, 1.7388],
          [1.6032, 2.7890, 2.5647, 2.1229, 2.5603, 1.6270],
          [2.3407, 3.4430, 3.3401, 2.8424, 3.5280, 1.9627]]],


        [[[1.7584, 1.6373, 1.7781, 2.0835, 2.0114, 2.3218],
          [2.0874, 1.6023, 2.1677, 1.5689, 1.9413, 2.7391],
          [1.9117, 1.8505, 1.7674, 1.7359, 1.6690, 2.8128],
          [0.9375, 1.0843, 1.4811, 0.9806, 0.9975, 1.1700]],

         [[2.1752, 3.0235, 2.3582, 1.6890, 2.5907, 2.0623],
          [2.1033, 2.5483, 1.6289, 1.6029, 1.9231, 1.4339],
          [2.8346, 3.0774, 2.4985, 1.7280, 2.7036, 2.2743],
          [2.0064, 3.0427, 2.5664, 1.8976, 2.3499, 2.1540]]]])

In [ ]:
def new_matmul1(a1,a2):

    k = a1.shape[0]*a1.shape[1]    
    mul_list = []
    for i in range(a1.shape[0]):
        for j in range(a1.shape[1]):
            
            mul = torch.matmul(a1[i,j,:],a2[i,j,:])                
            mul_list.append(mul)

    combine = torch.stack(mul_list)  
    combine = combine.view(a1.shape[0],a1.shape[1],combine.shape[1],combine.shape[2])
    
    return combine

result1 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result1.shape)
print(result1)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.7529, 1.8048, 3.4391, 3.1496, 2.6274, 3.4769],
          [2.4426, 2.0310, 2.9492, 2.6178, 2.0630, 3.0072],
          [2.5750, 1.7425, 2.7397, 2.7792, 2.1718, 3.1972],
          [1.6580, 1.1785, 2.3369, 1.8366, 1.7175, 2.1643]],

         [[1.8686, 2.8051, 1.5285, 2.3255, 2.7303, 1.7538],
          [1.8524, 3.0672, 2.7623, 2.5586, 2.9062, 1.7388],
          [1.6032, 2.7890, 2.5647, 2.1229, 2.5603, 1.6270],
          [2.3407, 3.4430, 3.3401, 2.8424, 3.5280, 1.9627]]],


        [[[1.7584, 1.6373, 1.7781, 2.0835, 2.0114, 2.3218],
          [2.0874, 1.6023, 2.1677, 1.5689, 1.9413, 2.7391],
          [1.9117, 1.8505, 1.7674, 1.7359, 1.6690, 2.8128],
          [0.9375, 1.0843, 1.4811, 0.9806, 0.9975, 1.1700]],

         [[2.1752, 3.0235, 2.3582, 1.6890, 2.5907, 2.0623],
          [2.1033, 2.5483, 1.6289, 1.6029, 1.9231, 1.4339],
          [2.8346, 3.0774, 2.4985, 1.7280, 2.7036, 2.2743],
          [2.0064, 3.0427, 2.5664, 1.8976, 2.3499, 2.1540]

In [ ]:
def new_matmul2(a1, a2):

    combine = torch.stack([torch.matmul(a1[i][j], a2[i][j]) for i in range(a1.shape[0]) for j in range(a1.shape[1])], dim=0)        
    return combine.reshape(a1.shape[0], a1.shape[1], a1.shape[2], a2.shape[3])

result2 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result2.shape)
print(result2)
print(torch.all(result==result2))

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.7529, 1.8048, 3.4391, 3.1496, 2.6274, 3.4769],
          [2.4426, 2.0310, 2.9492, 2.6178, 2.0630, 3.0072],
          [2.5750, 1.7425, 2.7397, 2.7792, 2.1718, 3.1972],
          [1.6580, 1.1785, 2.3369, 1.8366, 1.7175, 2.1643]],

         [[1.8686, 2.8051, 1.5285, 2.3255, 2.7303, 1.7538],
          [1.8524, 3.0672, 2.7623, 2.5586, 2.9062, 1.7388],
          [1.6032, 2.7890, 2.5647, 2.1229, 2.5603, 1.6270],
          [2.3407, 3.4430, 3.3401, 2.8424, 3.5280, 1.9627]]],


        [[[1.7584, 1.6373, 1.7781, 2.0835, 2.0114, 2.3218],
          [2.0874, 1.6023, 2.1677, 1.5689, 1.9413, 2.7391],
          [1.9117, 1.8505, 1.7674, 1.7359, 1.6690, 2.8128],
          [0.9375, 1.0843, 1.4811, 0.9806, 0.9975, 1.1700]],

         [[2.1752, 3.0235, 2.3582, 1.6890, 2.5907, 2.0623],
          [2.1033, 2.5483, 1.6289, 1.6029, 1.9231, 1.4339],
          [2.8346, 3.0774, 2.4985, 1.7280, 2.7036, 2.2743],
          [2.0064, 3.0427, 2.5664, 1.8976, 2.3499, 2.1540]

In [ ]:
def new_matmul3(a1, a2):
  '''
  a1 = (2,2,4,8)
  a2 = (2,2,6,8)
  torch(a2, a1.T)
  '''
  B1, N1, H1, T1 = a1.shape
  B2, N2, T2, H2 = a2.shape
  a1 = a1.reshape(B1 * N1 * H1, T1)
  #print('a1 : ', a1.shape)
  a2 = a2.permute(2,0,1,3)
  a2 = a2.reshape(T2, B2 * N2 * H2)
  #print('a2 : ', a2.shape)
  result = torch.matmul(a1, a2)
  result = torch.stack([result[i*H1:(i+1)*H1 ,i*H2:(i+1)*H2] for i in range(B1*N1)])
  result = result.reshape(B1,N1,H1,H2)
  return result

result3 = new_matmul3(a1, a2.transpose(-2,-1))
print('Shape : ', result3.shape)
print(result3)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.7529, 1.8048, 3.4391, 3.1496, 2.6274, 3.4769],
          [2.4426, 2.0310, 2.9492, 2.6178, 2.0630, 3.0072],
          [2.5750, 1.7425, 2.7397, 2.7792, 2.1718, 3.1972],
          [1.6580, 1.1785, 2.3369, 1.8366, 1.7175, 2.1643]],

         [[1.8686, 2.8051, 1.5285, 2.3255, 2.7303, 1.7538],
          [1.8524, 3.0672, 2.7623, 2.5586, 2.9062, 1.7388],
          [1.6032, 2.7890, 2.5647, 2.1229, 2.5603, 1.6270],
          [2.3407, 3.4430, 3.3401, 2.8424, 3.5280, 1.9627]]],


        [[[1.7584, 1.6373, 1.7781, 2.0835, 2.0114, 2.3218],
          [2.0874, 1.6023, 2.1677, 1.5689, 1.9413, 2.7391],
          [1.9117, 1.8505, 1.7674, 1.7359, 1.6690, 2.8128],
          [0.9375, 1.0843, 1.4811, 0.9806, 0.9975, 1.1700]],

         [[2.1752, 3.0235, 2.3582, 1.6890, 2.5907, 2.0623],
          [2.1033, 2.5483, 1.6289, 1.6029, 1.9231, 1.4339],
          [2.8346, 3.0774, 2.4985, 1.7280, 2.7036, 2.2743],
          [2.0064, 3.0427, 2.5664, 1.8976, 2.3499, 2.1540]

In [ ]:
def new_matmul3_plus(a1, a2):
    '''
    a1 = (2,2,4,8)
    a2 = (2,2,6,8)
    torch(a2, a1.T)
    '''
    B1, N1, H1, T1 = a1.shape
    B2, N2, T2, H2 = a2.shape
    #print(H1,T1,T2,H2)
    a1 = a1.reshape(B1 * N1 * H1, T1)

    a2 = a2.permute(2,0,1,3)
    a2 = a2.reshape(T2, B2 * N2 * H2)

    result = torch.matmul(a1, a2)
    result = result.reshape(B1*N1, B1*N1, H1, H2).transpose(1,2)
    result = result.reshape(B1*N1*B1*N1, H1, H2)

    index = torch.arange(0, B1*N1*B1*N1, (B1*N1)+1)
    #result = result[index]
    #result= torch.index_select(result,0, index)
    result = result[0:4,:,:]
    print(result.shape)
    #result = result.reshape(1, N1, H1, H2)
    result = result.unsqueeze(0)
    print(result.shape)

    return result


a1 = torch.rand(1,4,150,16)
a2 = torch.rand(1,4,10,16)
result3_plus = new_matmul3_plus(a1, a2.transpose(-2,-1))
print('Shape : ', result3_plus.shape)
print(result3_plus)   

torch.Size([4, 150, 10])
torch.Size([1, 4, 150, 10])
Shape :  torch.Size([1, 4, 150, 10])
tensor([[[[3.3838, 3.4120, 3.1763,  ..., 2.8965, 3.7930, 4.0286],
          [3.3731, 3.8181, 3.4525,  ..., 3.6661, 5.1197, 4.4346],
          [3.9121, 3.2665, 3.6328,  ..., 2.9576, 4.6977, 3.8945],
          ...,
          [2.4998, 3.4385, 3.3605,  ..., 3.1234, 4.2989, 4.4229],
          [4.0505, 3.9032, 3.7065,  ..., 3.2579, 4.0980, 4.0834],
          [3.5518, 4.2238, 3.7948,  ..., 2.9784, 3.0524, 3.3058]],

         [[2.8244, 3.6988, 3.1481,  ..., 2.4031, 3.4871, 3.2455],
          [3.7828, 5.2183, 4.0348,  ..., 3.7653, 3.7374, 3.7913],
          [3.0486, 3.5487, 3.0653,  ..., 3.7951, 4.5977, 4.5349],
          ...,
          [3.3623, 3.3315, 4.0794,  ..., 3.6210, 4.2255, 3.8217],
          [3.2363, 4.5231, 3.3725,  ..., 3.9814, 4.6853, 5.5462],
          [2.9573, 3.0535, 3.6217,  ..., 2.7151, 3.8101, 3.8697]],

         [[4.6520, 5.1243, 4.9299,  ..., 4.3121, 4.3224, 4.1277],
          [4.5314,

In [ ]:
def new_matmul4(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3])
  result = torch.matmul(a1_matrix, a2_matrix)
  result = torch.matmul(result, identity2)
  return result

def new_matmul_final(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  #identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  #identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3]).cuda()
  result = torch.matmul(a1_matrix, a2_matrix)
  #result = torch.matmul(result, identity2)
  return result

#result4 = new_matmul4(a1,a2.transpose(-2,-1))
#print('Shape : ', result4.shape)
#print(result4)

In [ ]:

B2, N2, T2, H2 = a2.shape
a2 = a2.reshape( B2 * N2* T2, H2)
print(a2.shape)
one = torch.ones(B2*N2, H2)
a2_I = torch.diag_embed(one).permute(1,0,2).reshape(H2, B2*N2*H2)
a2_m = torch.matmul(a2, a2_I)
a2_m

torch.Size([6, 4])


tensor([[0.7210, 0.2294, 0.4558, 0.0930, 0.7210, 0.2294, 0.4558, 0.0930],
        [0.2017, 0.0607, 0.4425, 0.3461, 0.2017, 0.0607, 0.4425, 0.3461],
        [0.5573, 0.3538, 0.4102, 0.7203, 0.5573, 0.3538, 0.4102, 0.7203],
        [0.8404, 0.7805, 0.1481, 0.0404, 0.8404, 0.7805, 0.1481, 0.0404],
        [0.7765, 0.7399, 0.3273, 0.6756, 0.7765, 0.7399, 0.3273, 0.6756],
        [0.1364, 0.3438, 0.1867, 0.5070, 0.1364, 0.3438, 0.1867, 0.5070]])

In [ ]:
one = torch.ones(B2*N2, H2)
a2_I = torch.diag_embed(one).permute(1,0,2).reshape(H2, B2*N2*H2)
a2_I

tensor([[1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1.]])

In [ ]:
#test = torch.eye(a2.shape[1])
one = torch.ones(2,4)
a2_I = torch.diag_embed(one)
a2_I
#test = torch.matmul(a2,test)
#test

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])

In [ ]:
a1 = torch.rand(1,4,15000,16).cuda()
a2 = torch.rand(1,4,1000,16).cuda()

%timeit -n 100 -r 7 torch.matmul(a1, a2.transpose(-2,-1))

25.4 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 100 -r 7 new_matmul1(a1, a2.transpose(-2,-1))

3.99 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 100 -r 7 new_matmul2(a1, a2.transpose(-2,-1))

3.57 ms ± 46.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 200 -r 7 new_matmul3(a1, a2.transpose(-2,-1))

The slowest run took 8.39 times longer than the fastest. This could mean that an intermediate result is being cached.
2.01 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 200 loops each)


In [ ]:
%timeit -n 100 -r 7 new_matmul4(a1, a2.transpose(-2,-1))

11.5 ms ± 7.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])

%timeit -n 100 -r 7 torch.matmul(a1_matrix, a2_matrix.transpose(-2,-1))

746 µs ± 36.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
from numpy import matrix
A = matrix([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0],[0,0,1]])
A2 = A.I
B = matrix([[1,3,2],[6,1,2],[3,1,1],[1,4,1],[1,1,2],[3,2,1]])
B * A2 * 2

matrix([[1., 3., 2., 1., 3., 2.],
        [6., 1., 2., 6., 1., 2.],
        [3., 1., 1., 3., 1., 1.],
        [1., 4., 1., 1., 4., 1.],
        [1., 1., 2., 1., 1., 2.],
        [3., 2., 1., 3., 2., 1.]])

In [ ]:
AB = matrix([[1,3,2,0,0,0],[6,1,2,0,0,0],[3,1,1,0,0,0],[0,0,0,1,4,1],[0,0,0,1,1,2],[0,0,0,3,2,1]])
I1 = matrix([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0],[0,0,1]])
I2 = matrix([[1,0,0,1,0,0],[0,1,0,0,1,0],[0,0,1,0,0,1]])
res = AB*I1
res

matrix([[1, 3, 2],
        [6, 1, 2],
        [3, 1, 1],
        [1, 4, 1],
        [1, 1, 2],
        [3, 2, 1]])

In [ ]:
x = torch.rand(1,10,64)
#print(x)
gamma, color = x[:, 0].unsqueeze(1), x[:, 1:]
#print(gamma)
#print(color)
#print(color.shape)
gamma_linear = nn.Linear(64, 1)
gamma_base = nn.Parameter(torch.ones((1)), requires_grad=False)
gamma = gamma_linear(gamma).squeeze(-1) 
#print(gamma_base)

color_linear = nn.Linear(64, 1)
color_base = nn.Parameter(torch.eye((3)), requires_grad=True)
color = color_linear(color).squeeze(-1).view(-1, 3, 3) + color_base
print(color)

tensor([[[ 0.7081, -0.1413,  0.0535],
         [-0.1295,  0.9730, -0.2034],
         [ 0.1608,  0.1183,  1.0428]]], grad_fn=<AddBackward0>)
